# CNN for classifying fake news

In [35]:
!pip install scikit-learn
!pip install keras
!pip install tensorflow
!pip install tf-keras
!pip install scikeras


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

In [11]:
df = pd.read_csv('../../data/processed_dataset.csv')

df.shape

(44898, 23)

In [17]:
X = df['text']
y = df['label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer

vocab_size = 5000
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)


In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


max_length = 100
padding_type = "post"
trunction_type="post"
X_train_padded = pad_sequences(X_train_sequences,maxlen=max_length, padding=padding_type,
                       truncating=trunction_type)
X_test_padded = pad_sequences(X_test_sequences,maxlen=max_length,
                               padding=padding_type, truncating=trunction_type)

In [21]:
!wget --no-check-certificate \
http://nlp.stanford.edu/data/glove.6B.zip \
-O /tmp/glove.6B.zip

--2025-04-23 00:45:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-04-23 00:45:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-04-23 00:45:34--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘/tmp/glove.6B.zip’


In [22]:
import os
import zipfile
with zipfile.ZipFile('/tmp/glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp/glove')

In [23]:
import numpy as np
embeddings_index = {}
f = open('/tmp/glove/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [24]:
embeddings_index.get("paper")

array([-0.85034  ,  0.33358  , -0.65889  , -0.49871  ,  0.36585  ,
       -0.19245  ,  0.25658  , -0.053408 ,  0.31474  ,  0.2443   ,
        0.29337  , -0.44917  ,  0.15175  ,  0.39314  , -0.31786  ,
        0.060525 ,  0.81775  , -0.38847  ,  0.76761  , -1.1041   ,
       -0.1544   ,  0.31655  , -0.37238  , -0.11485  ,  0.51635  ,
       -0.39289  ,  0.16301  , -0.2532   , -0.50976  ,  0.15201  ,
        0.27808  ,  0.52522  , -0.38815  , -0.3472   , -0.61818  ,
        0.17022  ,  0.12251  , -0.24191  , -0.38877  , -0.53176  ,
       -0.46987  , -0.70502  , -0.62126  , -0.38689  , -0.85637  ,
       -0.41003  , -0.47487  , -0.21083  , -0.81338  , -0.52398  ,
        0.49894  ,  0.37909  ,  0.55428  ,  1.123    , -0.42121  ,
       -1.5674   , -0.56892  ,  0.40819  ,  1.7949   ,  0.16856  ,
       -0.0029332,  0.28786  , -0.90088  , -0.094214 ,  0.79993  ,
       -0.39096  ,  0.76286  ,  0.71307  ,  0.13194  , -0.40756  ,
       -0.18687  ,  0.89562  ,  0.46867  , -0.0028801,  0.0253

In [25]:
word_index = tokenizer.word_index

embedding_matrix = np.zeros((len(word_index) + 1, max_length))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [26]:
embedding_matrix.shape

(125546, 100)

In [27]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(input_dim=len(word_index) + 1,
                            output_dim=max_length,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

/Users/ricardomendezcavalieri/Developer/work/IE/Y3/NLP/factos_api/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense

model = Sequential([
    embedding_layer,
  Conv1D(128, 5, activation='relu'),
  GlobalMaxPooling1D(),
  Dense(10, activation='relu'),
  Dense(1, activation='sigmoid')
])

In [29]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [31]:

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=0,  # Stop immediately when validation loss exceeds training loss
    restore_best_weights=True
)

history = model.fit(
    X_train_padded, 
    y_train,
    epochs=20,
    validation_data=(X_test_padded, y_test),
    callbacks=[early_stopping]
)

Epoch 1/20
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9999 - loss: 9.1373e-04 - val_accuracy: 0.9992 - val_loss: 0.0050
Epoch 2/20
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 1.0000 - loss: 8.6879e-05 - val_accuracy: 0.9992 - val_loss: 0.0052


In [32]:
loss, accuracy = model.evaluate(X_test_padded,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))

281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9993 - loss: 0.0031
Testing Accuracy is 99.92204904556274 


In [50]:
import itertools
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Dense
from sklearn.model_selection import train_test_split

# Split a validation set manually
X_train_part, X_val, y_train_part, y_val = train_test_split(
    X_train_padded, y_train, test_size=0.2, random_state=42
)

param_grid = {
    'num_filters': [128, 256],
    'kernel_size': [3, 5]
}

param_combinations = list(itertools.product(
    param_grid['num_filters'],
    param_grid['kernel_size']
))

results = []

for num_filters, kernel_size in param_combinations:
    print(f"\n🔧 Training with num_filters={num_filters}, kernel_size={kernel_size}")
    
    model = Sequential([
        embedding_layer,  # This must be defined earlier
        Conv1D(num_filters, kernel_size, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(10, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(
        X_train_part, y_train_part,
        epochs=10,
        batch_size=32,
        verbose=0,
        validation_data=(X_val, y_val)
    )
    
    val_acc = history.history['val_accuracy'][-1]
    print(f"✅ Validation Accuracy: {val_acc:.4f}")
    
    results.append({
        'num_filters': num_filters,
        'kernel_size': kernel_size,
        'val_accuracy': val_acc
    })

# Sort by best val accuracy
best_result = sorted(results, key=lambda x: x['val_accuracy'], reverse=True)[0]

print("\n🏆 Best combination:")
print(best_result)


🔧 Training with num_filters=128, kernel_size=3
✅ Validation Accuracy: 0.9989

🔧 Training with num_filters=128, kernel_size=5
✅ Validation Accuracy: 0.9985

🔧 Training with num_filters=256, kernel_size=3
✅ Validation Accuracy: 0.9986

🔧 Training with num_filters=256, kernel_size=5
✅ Validation Accuracy: 0.9983

🏆 Best combination:
{'num_filters': 128, 'kernel_size': 3, 'val_accuracy': 0.9988864064216614}


In [55]:
# Download the best model

# Save the best model
best_model = Sequential([
    embedding_layer,
    Conv1D(best_result['num_filters'], best_result['kernel_size'], activation='relu'),
    GlobalMaxPooling1D(),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])
best_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=0,
    restore_best_weights=True
)

best_model.fit(
    X_train_padded, 
    y_train,
    epochs=20,
    validation_data=(X_test_padded, y_test),
    callbacks=[early_stopping]
)

# Save the best model
best_model.save('best_model.keras')


Epoch 1/20
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.8897 - loss: 0.2202 - val_accuracy: 0.9984 - val_loss: 0.0060
Epoch 2/20
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9988 - loss: 0.0061 - val_accuracy: 0.9986 - val_loss: 0.0056
Epoch 3/20
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9996 - loss: 0.0019 - val_accuracy: 0.9991 - val_loss: 0.0030
Epoch 4/20
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9999 - loss: 4.5514e-04 - val_accuracy: 0.9991 - val_loss: 0.0037


In [56]:
from tensorflow import keras

# Load the best model
best_model = keras.models.load_model('best_model.keras')

# Evaluate the best model
loss, accuracy = best_model.evaluate(X_test_padded, y_test)
print(f"🔍 Testing Accuracy: {accuracy:.4f}")


281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9990 - loss: 0.0029
🔍 Testing Accuracy: 0.9991


In [ ]:
# Reporting the Precision, Recall, F1 Score, and Confusion Matrix
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Predict on the test set
y_pred = best_model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)

# Calculate all metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("\n📊 Model Performance Report")
print("=" * 50)
print(f"Precision:       {precision:.4f}")
print(f"Recall:         {recall:.4f}") 
print(f"F1 Score:       {f1:.4f}")
print("\nConfusion Matrix:")
print("-" * 20)
print(f"True Negatives:  {conf_matrix[0][0]}")
print(f"False Positives: {conf_matrix[0][1]}")
print(f"False Negatives: {conf_matrix[1][0]}")
print(f"True Positives:  {conf_matrix[1][1]}")
print("=" * 50)



281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

📊 Model Performance Report
Precision:       0.9993
Recall:         0.9988
F1 Score:       0.9991

Confusion Matrix:
--------------------
True Negatives:  4730
False Positives: 3
False Negatives: 5
True Positives:  4242


In [66]:
y_pred

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [70]:
# Use the model in an example

# Example usage
example_text = "The Federal Reserve announced yesterday that it will maintain current interest rates following its latest policy meeting. According to Chairperson Janet Powell, economic indicators show steady growth at 2.3% annually, while inflation remains within the target range of 2%. Several economists from major financial institutions have expressed support for this decision, citing recent employment data that shows unemployment holding steady at 3.6%. The Treasury Department is expected to release additional economic forecasts next week."
example_sequence = tokenizer.texts_to_sequences([example_text])
example_padded = pad_sequences(example_sequence, maxlen=max_length, padding='post')

# Predict the example
prediction = best_model.predict(example_padded)
predicted_label = 1 if prediction[0][0] >= 0.5 else 0
print(f"🔍 Raw Prediction Score: {prediction[0][0]:.4f}")
print(f"🏷️ Predicted Label: {predicted_label} ({'True' if predicted_label == 1 else 'False'})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
🔍 Raw Prediction Score: 0.0002
🏷️ Predicted Label: 0 (False)
